In [1]:
#dependencies
import pandas as pd
import pgeocode
from config import gkey
import requests
import json
import pymongo
import numpy as np
import time

<h1>How to Insert From an API Call</h1>
Data is pulled from the google maps api and inserted into MongoDB

In [2]:
#connect to mongo db and create "locations_mdb" database
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define the 'classDB' database in Mongo
db = client.locations_mdb

In [3]:
#Build the zip code reference table
zips_df=pd.read_csv('sf_zips.csv')
zips_df["Latitude"] = ""
zips_df["Longitude"] = ""
tool = pgeocode.Nominatim('US')

for index, row in zips_df.iterrows():
    zipcode = row['ZIPCode']
    tool_table=tool.query_postal_code(zipcode)
    lat=tool_table["latitude"]
    long=tool_table["longitude"]
    zips_df.loc[index, 'Latitude']=lat
    zips_df.loc[index, 'Longitude']=long
    

In [4]:
#HOTELS collection: build from google api data
#load into mongoDB

api_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in zips_df.iterrows():
    time.sleep(0.5)
    target_coordinates = f"{zips_df.loc[index, 'Latitude']},{zips_df.loc[index, 'Longitude']}" 
    target_radius = 50000
    target_type = "lodging"

# set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gkey}

# run a request using our params dictionary
    response = requests.get(api_url, params=params).json()
    
    
    db.hotels.insert_one(
        {
            'zip_code': row['ZIPCode'],
            'type': row['Type'],
            'county': row['County'],
            'population': row['Population'],
            'area_codes': row['Area Code(s)'],
            'latitude': row['Latitude'],
            'longitude': row['Longitude'],
            'hotels': response["results"]
        }
    )

In [39]:
#Build hotels database in PANDAS
hotels_df = pd.DataFrame(columns = ['Name', 'Address', "Status", 'Lattitude', "Longitude"]) 

api_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in zips_df.iterrows():
    time.sleep(0.5)
    target_coordinates = f"{zips_df.loc[index, 'Latitude']},{zips_df.loc[index, 'Longitude']}" 
    target_radius = 50000
    target_type = "lodging"

# set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gkey}

    response = requests.get(api_url, params=params).json()

# create a pandas dataframe of hotels
    for i in range(0,len(response["results"])):
        name=response["results"][i]["name"]
        lat=response["results"][i]["geometry"]["location"]["lat"]
        long=response["results"][i]["geometry"]["location"]["lng"]
        status=response["results"][i]["business_status"]
        address=response["results"][i]['vicinity']

        hotels_df = hotels_df.append({'Name' : name, 'Address' : address, 'Status' : status, "Lattitude" :lat, "Longitude": long},  
                ignore_index = True) 
    

In [57]:
#Drop any duplicates
hotels_df.drop_duplicates(keep='first', inplace=True, ignore_index=True)

In [56]:
    #Insert entries into MongoDB
    for index, row in hotels_df.iterrows():

        db.hotels.insert_one(
            {
                'Name': f"{hotels_df.loc[index, 'Name']}",
                'Address': f"{hotels_df.loc[index, 'Address']}",
                'Status': f"{hotels_df.loc[index, 'Status']}",
                'Lattitude': f"{hotels_df.loc[index, 'Lattitude']}",
                'Longitude': f"{hotels_df.loc[index, 'Longitude']}"

            }
        )

In [ ]:
#Retrieve your data